In [20]:
### 08.4. Output czytelny: 1 wiersz = 1 zdjęcie (zbiorcza tabela porównawcza)

"""
Tworzy tabelę porównawczą per-obraz:
- YOLO: lista subjectów PL/EN
- LVIS: lista subjectów PL/EN
- OCR:  pełny tekst (full_text)
- Human universal: ręczne sugestie (kolumna universal)
- CLIP scene: subject_pl / subject_en + scene_score (z outputs/csv/clip_scene_subjects.csv)

Wymaga: df_image_level (long) z kolumnami:
file_name, source_model, type, label_en, label_pl, text
"""

from __future__ import annotations
import pandas as pd
from pathlib import Path


def _join_unique(series: pd.Series, sep: str = "; ") -> str:
    vals = series.dropna().astype(str).map(lambda x: x.strip())
    vals = vals[vals.ne("") & vals.ne("nan")]
    uniq = pd.unique(vals)
    return sep.join(uniq.tolist())


def _load_clip_scene_best(path_csv: Path) -> pd.DataFrame:
    """
    Ładuje clip_scene_subjects.csv i wybiera najlepszą scenę per obraz (max scene_score).
    Oczekiwane kolumny: file_name, subject_en, subject_pl, scene_score
    Zwraca: file_name, scene_clip_pl, scene_clip_en, scene_clip_score
    """
    if not path_csv.exists():
        # brak pliku = brak kolumn sceny (nie wywalamy całej komórki)
        return pd.DataFrame(columns=["file_name", "scene_clip_pl", "scene_clip_en", "scene_clip_score"])

    df = pd.read_csv(path_csv)
    required = {"file_name", "subject_en", "subject_pl", "scene_score"}
    if not required.issubset(df.columns):
        raise ValueError(f"CLIP scene: oczekiwano {sorted(required)}, jest: {list(df.columns)}")

    df["file_name"] = df["file_name"].astype(str).map(lambda x: Path(x).name)

    best = (
        df.sort_values("scene_score", ascending=False)
          .drop_duplicates("file_name")
          .loc[:, ["file_name", "subject_pl", "subject_en", "scene_score"]]
          .rename(columns={
              "subject_pl": "scene_clip_pl",
              "subject_en": "scene_clip_en",
              "scene_score": "scene_clip_score",
          })
    )
    return best


def one_row_per_image(df_image_level: pd.DataFrame, clip_scene_csv: Path | None = None) -> pd.DataFrame:
    df = df_image_level.copy()

    # 1) Obiekty: agregacja (YOLO/LVIS)
    df_obj = df[df["type"] == "object"].copy()
    obj_agg = (
        df_obj.groupby(["file_name", "source_model"])
              .agg(
                  objects_pl=("label_pl", _join_unique),
                  objects_en=("label_en", _join_unique),
              )
              .reset_index()
    )

    obj_wide = obj_agg.pivot(index="file_name", columns="source_model", values=["objects_pl", "objects_en"])
    obj_wide.columns = [f"{a}__{b}" for a, b in obj_wide.columns]
    obj_wide = obj_wide.reset_index()

    # 2) Tekst: agregacja (OCR + human_universal)
    df_txt = df[df["type"] == "text"].copy()
    txt_agg = (
        df_txt.groupby(["file_name", "source_model"])["text"]
              .apply(lambda s: _join_unique(s, sep="\n---\n"))
              .reset_index(name="text_block")
    )
    txt_wide = txt_agg.pivot(index="file_name", columns="source_model", values="text_block").reset_index()

    # 3) Merge obiekty + tekst
    out = obj_wide.merge(txt_wide, on="file_name", how="outer")

    # 4) Merge CLIP scene (opcjonalnie)
    if clip_scene_csv is not None:
        df_scene_best = _load_clip_scene_best(clip_scene_csv)
        if not df_scene_best.empty:
            out = out.merge(df_scene_best, on="file_name", how="left")

    # 5) fillna tylko na kolumnach tekstowych
    for c in out.columns:
        if c != "file_name":
            out[c] = out[c].fillna("").astype(str)

    # 6) TWARDY wybór kolumn (docelowy układ)
    keep = [
        "file_name",
        "human_universal",
        "ocr",
        "scene_clip_pl",
        "scene_clip_en",
        "scene_clip_score",
        "objects_pl__yolo", "objects_en__yolo",
        "objects_pl__lvis", "objects_en__lvis",
    ]
    out = out.loc[:, [c for c in keep if c in out.columns]]

    return out


# --- OUTPUT + ZAPIS + PODGLĄD ---

OUT = Path("outputs/csv/08.4_image_level_comparison.csv")
OUT.parent.mkdir(parents=True, exist_ok=True)

P_CLIP_SCENE = Path("outputs/csv/clip_scene_subjects.csv")

df_per_image = one_row_per_image(df_image_level, clip_scene_csv=P_CLIP_SCENE)
df_per_image.to_csv(OUT, index=False)

print(OUT.as_posix())
df_per_image.head(20)

outputs/csv/08.4_image_level_comparison.csv


/var/folders/k2/5zl19j_12ps2q88dx76v08_c0000gp/T/ipykernel_3256/3466731979.py:63: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_obj.groupby(["file_name", "source_model"])
/var/folders/k2/5zl19j_12ps2q88dx76v08_c0000gp/T/ipykernel_3256/3466731979.py:78: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_txt.groupby(["file_name", "source_model"])["text"]


,file_name,human_universal,ocr,scene_clip_pl,scene_clip_en,scene_clip_score,objects_pl__yolo,objects_en__yolo,objects_pl__lvis,objects_en__lvis
0,0000.jpg,,a\nalamy\n0623-WK\nalamy\na\na\nalamy\nalamy\n...,auto,car exterior,0.018573572859168,autobus; samochód; osoba; ciężarówka,bus; car; person; truck,tablica rejestracyjna; reflektor; wycieraczka ...,license_plate; headlight; windshield_wiper; wh...
1,000202.jpg,,KOSZULE\nLIPSKA20,napisy lub hasła na murach,graffiti or political slogans on walls,0.0188813172280788,samochód; osoba; ciężarówka,car; person; truck,krawat; samochód; markiza; klamka; koło; latar...,necktie; car_(automobile); awning; doorknob; w...
2,000238.jpg,,HN-PO7,auto,car exterior,0.0193004831671714,samochód,car,wycieraczka samochodowa; pickup (samochód dost...,windshield_wiper; pickup_truck; headlight
3,0003.jpg,,alamy\nalamy\nalamy\nalamy\na\na\nalamy\nalamu...,auto,car exterior,0.0189944449812173,samochód; osoba; krawat; ciężarówka,car; person; tie; truck,krawat; kratka wentylacyjna; sweter; latarnia ...,necktie; vent; sweater; streetlight; license_p...
4,000345.jpg,,www.beskidia.pl,auto,car exterior,0.0188685730099678,samochód; osoba,car; person,wycieraczka samochodowa; kierownica; but z cho...,windshield_wiper; steering_wheel; boot; car_(a...
5,000346.jpg,,KASKADA,targowisko lub rynek,street market,0.0189518090337514,samochód; osoba; parasol,car; person; umbrella,tablica rejestracyjna; wycieraczka samochodowa...,license_plate; windshield_wiper; car_(automobi...
6,000388.jpg,,1,auto,car exterior,0.019241839647293,samochód,car,wycieraczka samochodowa; samochód; reflektor; ...,windshield_wiper; car_(automobile); headlight;...
7,000391.jpg,,Należy\nstaw\nwww.beskidia.pl,auto,car exterior,0.0190249178558588,osoba; ciężarówka,person; truck,kurtka; kapelusz; płaszcz; spódnica,jacket; hat; coat; skirt
8,0004.jpg,wnętrze sklepu,"ARCEY\n188\nCWIKŁAWIKŁA\nPARD\n12""\nCWIKŁ\nCWI...",wnętrze sklepu,shop interior,0.0189299695193767,butelka,bottle,zakrętka od butelki; butelka wina; etykieta; s...,bottle_cap; wine_bottle; tag; signboard; bottle
9,000406.jpg,,,auto,car exterior,0.0195006616413593,samochód; ciężarówka,car; truck,kierownica; kratka wentylacyjna; klamka; śruba...,steering_wheel; vent; doorknob; bolt; car_(aut...
